## FPL GAFFER TOOL CALLING
This notebook contains experiments with boilerplate tools, showing how FPL Gaffer would chose the tools to call based on prompts from a user during conversations.

In [10]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

import os
import asyncio
import json
from typing import List, Literal, Callable, Any
from pydantic import BaseModel, Field, ConfigDict
from langchain.tools import Tool, BaseTool
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from IPython.display import display, Image

In [2]:
# TODO: Use boilerplate functions for tools, assess
# with an LLM to test the tools that would be called
# for different kinds of prompts.

# NOTE: Use proper documentation for the notebook.

In [7]:
class AsyncFPLTool(BaseTool):
    """Custom async tool wrapper to handle async execution in LangGraph context."""
    func: Callable
    input_schema: BaseModel = None

    # Allow custom types
    model_config = ConfigDict(arbitrary_types_allowed=True)

    async def _arun(self, **kwargs) -> Any:
        """Async execution of the tool."""
        try:
            return await self.func(**kwargs)
        except Exception as e:
            return {"error": e}

    def _run(self, **kwargs) -> Any:
        """Fallback sync wrapper."""
        try:
            return asyncio.run(self.func(**kwargs))
        except Exception as e:
            return {"error": e}

# Tool input schemas
class NewsSearchInput(BaseModel):
    """Input schema for the news search tool."""
    query: str = Field(..., description="The search query for news search.")

class PlayerByPositionInput(BaseModel):
    """Input schema for the player by position tool."""
    position: Literal["GKP", "DEF", "MID", "FWD"] = Field(..., description="Position to search for. One of: GK, DEF, MID, FWD.")
    max_price: float = Field(15.0, description="Maximum player price to search for in millions.")


class PlayerDataInput(BaseModel):
    """Input schema for the player data tool."""
    player_names: List[str] = Field(..., description="List of player(s) names to fetch data for.")


class FixturesForRangeInput(BaseModel):
    """Input schema for the fixtures for range tool."""
    num_gameweeks: int = Field(..., description="Number of upcoming gameweeks to fetch fixtures for.")

class UserTeamInfoInput(BaseModel):
    """Input schema for the user team info tool."""
    manager_id: int = Field(..., description="The user's FPL manager ID.")
    gameweek: int = Field(..., description="The current gameweek number to fetch data for.")

/home/d3ju/Documents/fpl-gaffer/.venv/lib64/python3.13/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "input_schema" in "AsyncFPLTool" shadows an attribute in parent "BaseTool"
  warnings.warn(


In [32]:
# Boilerplate tool functions
def news_search_tool(query: str):
    print(f"[news_search_tool] query: {query}")

def get_user_team_info_tool(manager_id: int, gameweek: int):
    print(f"[get_user_team_info_tool] manager_id={manager_id}, gameweek={gameweek}")

def get_players_by_position_tool(position: Literal["GKP", "DEF", "MID", "FWD"], max_price: float):
    print(f"[get_players_by_position_tool] position={position}, max_price={max_price}")

def get_player_data_tool(player_names: List[str]):
    print(f"[get_player_data_tool] player_names={player_names}")

def get_fixtures_for_range_tool(num_gameweeks: int):
    print(f"[get_fixtures_for_range_tool] num_gameweeks={num_gameweeks}")

# Create tool list
tools = [
        AsyncFPLTool(
            name="news_search_tool",
            description="Search for FPL news, expert analysis, injury updates, press conference information, etc."
                        "Use this when you need information about player/team news, injury, expert opinions, or "
                        "general FPL updates.",
            func=news_search_tool,
            args_schema=NewsSearchInput
        ),
        AsyncFPLTool(
            name="get_user_team_info_tool",
            description="Get comprehensive information about a user's FPL team including squad, transfers, "
                        "and finances. Use this when you need information about the user's team, players, or "
                        "financial situation.",
            func=get_user_team_info_tool,
            args_schema=UserTeamInfoInput
        ),
        AsyncFPLTool(
            name="get_players_by_position_tool",
            description="Get players by position and max price. Use this when you need information for player "
                        "replacements or transfer suggestions based on position and budget.",
            func=get_players_by_position_tool,
            args_schema=PlayerByPositionInput
        ),
        AsyncFPLTool(
            name="get_player_data_tool",
            description="Get detailed player data including stats, form, and injuries. Use this when you need "
                        "information about specific players.",
            func=get_player_data_tool,
            args_schema=PlayerDataInput
        ),
        AsyncFPLTool(
            name="get_fixtures_for_range_tool",
            description="Get fixtures from the current gameweek to the next x gameweeks. Use this when you need "
                        "information about upcoming fixtures or planning for future gameweeks.",
            func=get_fixtures_for_range_tool,
            args_schema=FixturesForRangeInput
        )
    ]

TOOLS_DESCRIPTION = "\n".join(
    f"{t.name}: {t.description}" for t in tools
)

In [ ]:
MESSAGE_ANALYSIS_PROMPT_TEMPLATE = """
You are an FPL conversational assistant that needs to decide the tools to call to assist the user's query.


Available tools:
{tools}

Determine which tools to call and in what order. Do NOT provide any extra text, and do not add tools
that are not listed above.

Ouput MUST be one of:
1. For a single tool call, a List of the tool name like so:
["<tool_name>"]

2. For multiple tool calls, a LIST of JSON objects with tool names and arguments
like so: [{{ "name": "<name>" }}, {{ "name": "<name>" }}]

3.If no tool fits, respond with:
{{"name": "none", "message": "<natural language reply>" }}
"""

prompt = PromptTemplate.from_template(MESSAGE_ANALYSIS_PROMPT_TEMPLATE)
prompt = prompt.format(tools=tools)

print(prompt)


You are an FPL conversational assistant that needs to decide the tools to call to assist the user's query.


Available tools:
[AsyncFPLTool(name='news_search_tool', description='Search for FPL news, expert analysis, injury updates, press conference information, etc.Use this when you need information about player/team news, injury, expert opinions, or general FPL updates.', args_schema=<class '__main__.NewsSearchInput'>, func=<function news_search_tool at 0x7f0c48ecefc0>), AsyncFPLTool(name='get_user_team_info_tool', description="Get comprehensive information about a user's FPL team including squad, transfers, and finances. Use this when you need information about the user's team, players, or financial situation.", args_schema=<class '__main__.UserTeamInfoInput'>, func=<function get_user_team_info_tool at 0x7f0c48ecf060>), AsyncFPLTool(name='get_players_by_position_tool', description='Get players by position and max price. Use this when you need information for player replacements or t

In [ ]:
# TODO: Test running the tools directly to make this a single node
# instead of tool selection and execution nodes.

In [7]:
class ToolResponse(BaseModel):
    tools_to_call: List[str] = Field(..., description="The list of tools to call")

In [65]:
# Initialize llm
llm = ChatGroq(
    api_key=os.environ["GROQ_API_KEY"],
    model="llama-3.3-70b-versatile",
    temperature=0.4
)

In [66]:
def run_query(q: str):
    messages = [prompt, HumanMessage(content=q)]
    result = llm.invoke(messages)

    return result

In [67]:
run_query("Is Salah injured?")

AIMessage(content='[{"name": "news_search_tool"}]', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 590, 'total_tokens': 601, 'completion_time': 0.035928076, 'prompt_time': 0.051373182, 'queue_time': 0.084823585, 'total_time': 0.087301258}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--4fadd4f1-96d2-4e02-8465-6a1df5a67bff-0', usage_metadata={'input_tokens': 590, 'output_tokens': 11, 'total_tokens': 601})

In [68]:
response = run_query("Who should I captain this gameweek?")

In [69]:
print(response.content)

[{"name": "news_search_tool"}, {"name": "get_player_data_tool"}, {"name": "get_fixtures_for_range_tool"}]


In [70]:
run_query("Do you suggest I change any player from my team?")

AIMessage(content='[{"name": "get_user_team_info_tool"}, {"name": "get_player_data_tool"}, {"name": "get_players_by_position_tool"}]', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 597, 'total_tokens': 629, 'completion_time': 0.077542609, 'prompt_time': 0.052808529, 'queue_time': 0.085069839, 'total_time': 0.130351138}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ce5091bb-b9a8-4543-9a23-7d038cf5f5f5-0', usage_metadata={'input_tokens': 597, 'output_tokens': 32, 'total_tokens': 629})

In [71]:
response = run_query("Based on fixture difficuly in the next 5 gameweeks, what players do you think I should sell, and what teams can I invest in?")

In [72]:
response

AIMessage(content='[{"name": "get_fixtures_for_range_tool"}, {"name": "get_player_data_tool"}, {"name": "news_search_tool"}]', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 617, 'total_tokens': 648, 'completion_time': 0.070131387, 'prompt_time': 0.050517013, 'queue_time': 0.100716109, 'total_time': 0.1206484}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f62c95a0-b642-45f8-aff2-014e7e79d766-0', usage_metadata={'input_tokens': 617, 'output_tokens': 31, 'total_tokens': 648})